# 히스토리까지

In [5]:
import pandas as pd

data = []

stay_id_list = df['stay_id'].unique()

for stay_id in tqdm(stay_id_list):
    stay_data = df[df['stay_id'] == stay_id]
    
    time_range = pd.date_range(start=stay_data['time'].min() + pd.Timedelta(hours=1), end=stay_data['time'].max(), freq='H')
    
    for time in time_range:
        con_data = stay_data[(stay_data['time'] > time - pd.Timedelta(hours=1)) & (stay_data['time'] <= time)]
        
        con_data_1 = stay_data[(stay_data['time'] > time - pd.Timedelta(hours=2)) & (stay_data['time'] <= time- pd.Timedelta(hours=1))]
        
        vital = ['temperature', 'heartrate', 'resprate', 'o2sat', 'sbp', 'dbp']
        
        drug = df.columns[19:43] 
        
        aggregated_metrics = [con_data[metric].max() for metric in vital] + [con_data[metric].min() for metric in vital] + [con_data[metric].mean() for metric in vital]
        
        aggregated_metrics_1 = [con_data_1[metric].max() for metric in vital] + [con_data_1[metric].min() for metric in vital] + [con_data_1[metric].mean() for metric in vital]
        
        aggregated_metrics_2 = [con_data[metric].max() for metric in drug]
            
        aggregated_metrics_3 = [con_data_1[metric].max() for metric in drug]
        
        data.append((stay_id, time, *aggregated_metrics, *aggregated_metrics_1,*aggregated_metrics_2,*aggregated_metrics_3))

result_columns = ['stay_id', 'time', 'max_temp', 'max_hr', 'max_resp', 'max_o2sat', 'max_sbp', 'max_dbp', 'min_temp', 'min_hr', 'min_resp', 'min_o2sat', 'min_sbp', 'min_dbp', 'avg_temp', 'avg_hr', 'avg_resp', 'avg_o2sat', 'avg_sbp', 'avg_dbp','before_max_temp', 'before_max_hr', 'before_max_resp', 'before_max_o2sat', 'before_max_sbp', 'before_max_dbp', 'before_min_temp', 'before_min_hr', 'before_min_resp', 'before_min_o2sat', 'before_min_sbp', 'before_min_dbp', 'before_avg_temp', 'before_avg_hr', 'before_avg_resp', 'before_avg_o2sat', 'before_avg_sbp', 'before_avg_dbp']+ list(df.columns[19:43])+modified_list

result_df = pd.DataFrame(data, columns=result_columns)


100%|██████████| 282903/282903 [1:15:23<00:00, 62.53it/s]


# ########### ################################

In [1]:
import pandas as pd
pd.set_option('display.max_columns',None)
from datetime import timedelta
import pickle
import numpy as np
from tqdm import tqdm

In [2]:
df=pd.read_parquet("pre_ED_v0.1_240308.parquet")
df

,stay_id,time,gender,race,arrival_transport,result,temperature,heartrate,resprate,o2sat,sbp,dbp,acuity,chief_abdpain,chief_transfer,chief_chestpain,chief_pfall,chief_dyspnea,chief_fever,chief_headache,chief_backpain,chief_weakness,chief_dizziness,chief_cough,chief_woundeval,chief_mvc,chief_etoh,chief_altered_mental_status,first_time,last_time,drug_acetaminophen,drug_morphine,drug_ondansetron,drug_hydromorphone,drug_oxycodone,drug_lidocaine,drug_ibuprofen,drug_lorazepam,drug_albuterol,drug_ketorolac,drug_ceftriaxone,drug_ipratropium,drug_vancomycin,drug_bromide,drug_diazepam,drug_aspirin,drug_sodium,drug_magnesium,drug_fentanyl,drug_diphenhydramine,Comorbidity_AIDS_H1V,Comorbidity_Alcohol_abuse,Comorbidity_Blood_loss_anemia,Comorbidity_Chronic_pulmonary_disease,Comorbidity_Coagulopathy,Comorbidity_Deficiency_anemia,Comorbidity_Depression,Comorbidity_Diabetes_complicated,Comorbidity_Diabetes_uncomplicated,Comorbidity_Drug_abuse,Comorbidity_Fluid_electrolyte_disorders,Comorbidity_Hypothyroidism,Comorbidity_Liver_disease,Comorbidity_Lymphoma,Comorbidity_Metastatic_cancer,Comorbidity_Obesity,Comorbidity_Other_neurological_disorders,Comorbidity_Paralysis,Comorbidity_Pepticulcer_disease,Comorbidity_Peripheral_vascular,Comorbidity_Psychoses,Comorbidity_Pulmonary_circulation,Comorbidity_Renal_failure,Comorbidity_Rheumatoid_arthritis_collagen_vascular_diseases,Comorbidity_Solid_tumor_without_metastasis,Comorbidity_Weight_loss,Comorbidity_cardiac,Comorbidity_chf,Comorbidity_hypertension_complicated,Comorbidity_hypertension_uncomplicated,Comorbidity_valvular_disease,time_diff
0,30000012,2126-02-14 20:22:00,F,White,AMBULANCE,hos,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2126-02-14 20:22:00,2126-02-15 01:59:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaT
1,30000012,2126-02-14 20:22:00,None,None,None,None,37.111111,96.0,18.0,93.0,160.0,54.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2126-02-14 20:22:00,2126-02-15 01:59:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0 days 00:00:00
2,30000012,2126-02-14 20:23:00,None,None,None,None,37.111111,96.0,18.0,93.0,160.0,54.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2126-02-14 20:22:00,2126-02-15 01:59:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0 days 00:01:00
3,30000012,2126-02-14 23:43:00,None,None,None,None,NaN,80.0,13.0,99.0,112.0,44.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2126-02-14 20:22:00,2126-02-15 01:59:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0 days 03:20:00
4,30000012,2126-02-15 00:50:00,None,None,None,None,37.000000,88.0,16.0,100.0,135.0,51.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2126-02-14 20:22:00,2126-02-15 01:59:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0 days 01:07:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2407617,39999961,2145-05-16 19:56:00,None,None,No

In [3]:
df.columns[2:6].tolist()+df.columns[12:30].tolist()+df.columns[50:-1].tolist()

['gender',
 'race',
 'arrival_transport',
 'result',
 'acuity',
 'chief_abdpain',
 'chief_transfer',
 'chief_chestpain',
 'chief_pfall',
 'chief_dyspnea',
 'chief_fever',
 'chief_headache',
 'chief_backpain',
 'chief_weakness',
 'chief_dizziness',
 'chief_cough',
 'chief_woundeval',
 'chief_mvc',
 'chief_etoh',
 'chief_altered_mental_status',
 'first_time',
 'last_time',
 'Comorbidity_AIDS_H1V',
 'Comorbidity_Alcohol_abuse',
 'Comorbidity_Blood_loss_anemia',
 'Comorbidity_Chronic_pulmonary_disease',
 'Comorbidity_Coagulopathy',
 'Comorbidity_Deficiency_anemia',
 'Comorbidity_Depression',
 'Comorbidity_Diabetes_complicated',
 'Comorbidity_Diabetes_uncomplicated',
 'Comorbidity_Drug_abuse',
 'Comorbidity_Fluid_electrolyte_disorders',
 'Comorbidity_Hypothyroidism',
 'Comorbidity_Liver_disease',
 'Comorbidity_Lymphoma',
 'Comorbidity_Metastatic_cancer',
 'Comorbidity_Obesity',
 'Comorbidity_Other_neurological_disorders',
 'Comorbidity_Paralysis',
 'Comorbidity_Pepticulcer_disease',
 'Comor

In [4]:
static_var=df.columns[2:6].tolist()+df.columns[12:30].tolist()+df.columns[50:-1].tolist()

In [5]:
df[static_var] = df.groupby('stay_id')[static_var].ffill().bfill()

In [6]:
df

,stay_id,time,gender,race,arrival_transport,result,temperature,heartrate,resprate,o2sat,sbp,dbp,acuity,chief_abdpain,chief_transfer,chief_chestpain,chief_pfall,chief_dyspnea,chief_fever,chief_headache,chief_backpain,chief_weakness,chief_dizziness,chief_cough,chief_woundeval,chief_mvc,chief_etoh,chief_altered_mental_status,first_time,last_time,drug_acetaminophen,drug_morphine,drug_ondansetron,drug_hydromorphone,drug_oxycodone,drug_lidocaine,drug_ibuprofen,drug_lorazepam,drug_albuterol,drug_ketorolac,drug_ceftriaxone,drug_ipratropium,drug_vancomycin,drug_bromide,drug_diazepam,drug_aspirin,drug_sodium,drug_magnesium,drug_fentanyl,drug_diphenhydramine,Comorbidity_AIDS_H1V,Comorbidity_Alcohol_abuse,Comorbidity_Blood_loss_anemia,Comorbidity_Chronic_pulmonary_disease,Comorbidity_Coagulopathy,Comorbidity_Deficiency_anemia,Comorbidity_Depression,Comorbidity_Diabetes_complicated,Comorbidity_Diabetes_uncomplicated,Comorbidity_Drug_abuse,Comorbidity_Fluid_electrolyte_disorders,Comorbidity_Hypothyroidism,Comorbidity_Liver_disease,Comorbidity_Lymphoma,Comorbidity_Metastatic_cancer,Comorbidity_Obesity,Comorbidity_Other_neurological_disorders,Comorbidity_Paralysis,Comorbidity_Pepticulcer_disease,Comorbidity_Peripheral_vascular,Comorbidity_Psychoses,Comorbidity_Pulmonary_circulation,Comorbidity_Renal_failure,Comorbidity_Rheumatoid_arthritis_collagen_vascular_diseases,Comorbidity_Solid_tumor_without_metastasis,Comorbidity_Weight_loss,Comorbidity_cardiac,Comorbidity_chf,Comorbidity_hypertension_complicated,Comorbidity_hypertension_uncomplicated,Comorbidity_valvular_disease,time_diff
0,30000012,2126-02-14 20:22:00,F,White,AMBULANCE,hos,NaN,NaN,NaN,NaN,NaN,NaN,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2126-02-14 20:22:00,2126-02-15 01:59:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaT
1,30000012,2126-02-14 20:22:00,F,White,AMBULANCE,hos,37.111111,96.0,18.0,93.0,160.0,54.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2126-02-14 20:22:00,2126-02-15 01:59:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0 days 00:00:00
2,30000012,2126-02-14 20:23:00,F,White,AMBULANCE,hos,37.111111,96.0,18.0,93.0,160.0,54.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2126-02-14 20:22:00,2126-02-15 01:59:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0 days 00:01:00
3,30000012,2126-02-14 23:43:00,F,White,AMBULANCE,hos,NaN,80.0,13.0,99.0,112.0,44.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2126-02-14 20:22:00,2126-02-15 01:59:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0 days 03:20:00
4,30000012,2126-02-15 00:50:00,F,White,AMBULANCE,hos,37.000000,88.0,16.0,100.0,135.0,51.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2126-02-14 20:22:00,2126-02-15 01:59:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0 days 01:07:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2407617,39999961,2145-05-16 19:56:00,F,Bl

In [7]:
df.columns[30:50]

Index(['drug_acetaminophen', 'drug_morphine', 'drug_ondansetron',
       'drug_hydromorphone', 'drug_oxycodone', 'drug_lidocaine',
       'drug_ibuprofen', 'drug_lorazepam', 'drug_albuterol', 'drug_ketorolac',
       'drug_ceftriaxone', 'drug_ipratropium', 'drug_vancomycin',
       'drug_bromide', 'drug_diazepam', 'drug_aspirin', 'drug_sodium',
       'drug_magnesium', 'drug_fentanyl', 'drug_diphenhydramine'],
      dtype='object')

In [8]:
time_drug_var=df.columns[30:50]

In [9]:
df[time_drug_var].fillna(0, inplace=True)

/tmp/ipykernel_2479917/183064568.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[time_drug_var].fillna(0, inplace=True)


In [10]:
#df.to_csv("pre_ED_v0.2_231221.csv",index=0)

In [11]:
df

,stay_id,time,gender,race,arrival_transport,result,temperature,heartrate,resprate,o2sat,sbp,dbp,acuity,chief_abdpain,chief_transfer,chief_chestpain,chief_pfall,chief_dyspnea,chief_fever,chief_headache,chief_backpain,chief_weakness,chief_dizziness,chief_cough,chief_woundeval,chief_mvc,chief_etoh,chief_altered_mental_status,first_time,last_time,drug_acetaminophen,drug_morphine,drug_ondansetron,drug_hydromorphone,drug_oxycodone,drug_lidocaine,drug_ibuprofen,drug_lorazepam,drug_albuterol,drug_ketorolac,drug_ceftriaxone,drug_ipratropium,drug_vancomycin,drug_bromide,drug_diazepam,drug_aspirin,drug_sodium,drug_magnesium,drug_fentanyl,drug_diphenhydramine,Comorbidity_AIDS_H1V,Comorbidity_Alcohol_abuse,Comorbidity_Blood_loss_anemia,Comorbidity_Chronic_pulmonary_disease,Comorbidity_Coagulopathy,Comorbidity_Deficiency_anemia,Comorbidity_Depression,Comorbidity_Diabetes_complicated,Comorbidity_Diabetes_uncomplicated,Comorbidity_Drug_abuse,Comorbidity_Fluid_electrolyte_disorders,Comorbidity_Hypothyroidism,Comorbidity_Liver_disease,Comorbidity_Lymphoma,Comorbidity_Metastatic_cancer,Comorbidity_Obesity,Comorbidity_Other_neurological_disorders,Comorbidity_Paralysis,Comorbidity_Pepticulcer_disease,Comorbidity_Peripheral_vascular,Comorbidity_Psychoses,Comorbidity_Pulmonary_circulation,Comorbidity_Renal_failure,Comorbidity_Rheumatoid_arthritis_collagen_vascular_diseases,Comorbidity_Solid_tumor_without_metastasis,Comorbidity_Weight_loss,Comorbidity_cardiac,Comorbidity_chf,Comorbidity_hypertension_complicated,Comorbidity_hypertension_uncomplicated,Comorbidity_valvular_disease,time_diff
0,30000012,2126-02-14 20:22:00,F,White,AMBULANCE,hos,NaN,NaN,NaN,NaN,NaN,NaN,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2126-02-14 20:22:00,2126-02-15 01:59:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaT
1,30000012,2126-02-14 20:22:00,F,White,AMBULANCE,hos,37.111111,96.0,18.0,93.0,160.0,54.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2126-02-14 20:22:00,2126-02-15 01:59:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0 days 00:00:00
2,30000012,2126-02-14 20:23:00,F,White,AMBULANCE,hos,37.111111,96.0,18.0,93.0,160.0,54.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2126-02-14 20:22:00,2126-02-15 01:59:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0 days 00:01:00
3,30000012,2126-02-14 23:43:00,F,White,AMBULANCE,hos,NaN,80.0,13.0,99.0,112.0,44.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2126-02-14 20:22:00,2126-02-15 01:59:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0 days 03:20:00
4,30000012,2126-02-15 00:50:00,F,White,AMBULANCE,hos,37.000000,88.0,16.0,100.0,135.0,51.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2126-02-14 20:22:00,2126-02-15 01:59:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0 days 01:07:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2407617,39999961,2145-05-16 19:56:00,F,Bl

In [12]:
df['time'] = pd.to_datetime(df['time'])

In [13]:
#modified_list = ['before_' + item for item in list(df.columns[20:40])]

In [14]:
#modified_list

In [15]:
#sample_df=df.head(30)

# 현재 정보만

In [16]:
import pandas as pd

data = []

stay_id_list = df['stay_id'].unique()

for stay_id in tqdm(stay_id_list):
    stay_data = df[df['stay_id'] == stay_id]
    
    time_range = pd.date_range(start=stay_data['time'].min() + pd.Timedelta(hours=1), end=stay_data['time'].max(), freq='H')
    
    for time in time_range:
        con_data = stay_data[(stay_data['time'] >= time - pd.Timedelta(hours=1)) & (stay_data['time'] < time)]
        
        #con_data_1 = stay_data[(stay_data['time'] > time - pd.Timedelta(hours=2)) & (stay_data['time'] <= time- pd.Timedelta(hours=1))]
        
        vital = ['temperature', 'heartrate', 'resprate', 'o2sat', 'sbp', 'dbp']
        
        drug = df.columns[30:50]
        
        aggregated_metrics = [con_data[metric].max() for metric in vital] + [con_data[metric].min() for metric in vital] + [con_data[metric].mean() for metric in vital]
        
        #aggregated_metrics_1 = [con_data_1[metric].max() for metric in vital] + [con_data_1[metric].min() for metric in vital] + [con_data_1[metric].mean() for metric in vital]
        
        aggregated_metrics_2 = [con_data[metric].max() for metric in drug]
            
        #aggregated_metrics_3 = [con_data_1[metric].max() for metric in drug]
        
        data.append((stay_id, time, *aggregated_metrics,*aggregated_metrics_2))

result_columns = ['stay_id', 'time', 'max_temp', 'max_hr', 'max_resp', 'max_o2sat', 'max_sbp', 'max_dbp', 'min_temp', 'min_hr', 'min_resp', 'min_o2sat', 'min_sbp', 'min_dbp', 'avg_temp', 'avg_hr', 'avg_resp', 'avg_o2sat', 'avg_sbp', 'avg_dbp',]+ list(df.columns[30:50])
result_df = pd.DataFrame(data, columns=result_columns)

100%|██████████| 327954/327954 [1:04:33<00:00, 84.67it/s] 


In [17]:
result_df

,stay_id,time,max_temp,max_hr,max_resp,max_o2sat,max_sbp,max_dbp,min_temp,min_hr,min_resp,min_o2sat,min_sbp,min_dbp,avg_temp,avg_hr,avg_resp,avg_o2sat,avg_sbp,avg_dbp,drug_acetaminophen,drug_morphine,drug_ondansetron,drug_hydromorphone,drug_oxycodone,drug_lidocaine,drug_ibuprofen,drug_lorazepam,drug_albuterol,drug_ketorolac,drug_ceftriaxone,drug_ipratropium,drug_vancomycin,drug_bromide,drug_diazepam,drug_aspirin,drug_sodium,drug_magnesium,drug_fentanyl,drug_diphenhydramine
0,30000012,2126-02-14 21:22:00,37.111111,96.0,18.0,93.0,160.0,54.0,37.111111,96.0,18.0,93.0,160.0,54.0,37.111111,96.0,18.0,93.0,160.0,54.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,30000012,2126-02-14 22:22:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,30000012,2126-02-14 23:22:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,30000012,2126-02-15 00:22:00,NaN,80.0,13.0,99.0,112.0,44.0,NaN,80.0,13.0,99.0,112.0,44.0,NaN,80.0,13.0,99.0,112.0,44.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,30000012,2126-02-15 01:22:00,37.000000,88.0,16.0,100.0,135.0,51.0,37.000000,88.0,16.0,100.0,135.0,51.0,37.000000,88.0,16.0,100.0,135.0,51.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1777442,39999961,2145-05-16 21:16:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1777443,39999961,2145-05-16 22:16:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1777444,39999961,2145-05-16 23:16:00,36.666667,107.0,20.0,99.0,139.0,90.0,36.666667,107.0,20.0,99.0,139.0,90.0,36.666667,107.0,20.0,99.0,139.0,90.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1777445,39999961,2145-05-17 00:16:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
result_df.iloc[:, 20:]

,drug_acetaminophen,drug_morphine,drug_ondansetron,drug_hydromorphone,drug_oxycodone,drug_lidocaine,drug_ibuprofen,drug_lorazepam,drug_albuterol,drug_ketorolac,drug_ceftriaxone,drug_ipratropium,drug_vancomycin,drug_bromide,drug_diazepam,drug_aspirin,drug_sodium,drug_magnesium,drug_fentanyl,drug_diphenhydramine
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1777442,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1777443,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1777444,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1777445,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
result_df.iloc[:, 20:] = result_df.iloc[:, 20:].fillna(0)

In [20]:
result_df

,stay_id,time,max_temp,max_hr,max_resp,max_o2sat,max_sbp,max_dbp,min_temp,min_hr,min_resp,min_o2sat,min_sbp,min_dbp,avg_temp,avg_hr,avg_resp,avg_o2sat,avg_sbp,avg_dbp,drug_acetaminophen,drug_morphine,drug_ondansetron,drug_hydromorphone,drug_oxycodone,drug_lidocaine,drug_ibuprofen,drug_lorazepam,drug_albuterol,drug_ketorolac,drug_ceftriaxone,drug_ipratropium,drug_vancomycin,drug_bromide,drug_diazepam,drug_aspirin,drug_sodium,drug_magnesium,drug_fentanyl,drug_diphenhydramine
0,30000012,2126-02-14 21:22:00,37.111111,96.0,18.0,93.0,160.0,54.0,37.111111,96.0,18.0,93.0,160.0,54.0,37.111111,96.0,18.0,93.0,160.0,54.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,30000012,2126-02-14 22:22:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,30000012,2126-02-14 23:22:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,30000012,2126-02-15 00:22:00,NaN,80.0,13.0,99.0,112.0,44.0,NaN,80.0,13.0,99.0,112.0,44.0,NaN,80.0,13.0,99.0,112.0,44.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,30000012,2126-02-15 01:22:00,37.000000,88.0,16.0,100.0,135.0,51.0,37.000000,88.0,16.0,100.0,135.0,51.0,37.000000,88.0,16.0,100.0,135.0,51.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1777442,39999961,2145-05-16 21:16:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1777443,39999961,2145-05-16 22:16:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1777444,39999961,2145-05-16 23:16:00,36.666667,107.0,20.0,99.0,139.0,90.0,36.666667,107.0,20.0,99.0,139.0,90.0,36.666667,107.0,20.0,99.0,139.0,90.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1777445,39999961,2145-05-17 00:16:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [553]:
#result_df['inpute_max_temp'] = result_df.apply(lambda row: 1 if pd.isna(row['max_temp']) else 0, axis=1)

In [21]:
list(result_df.columns[2:20])

['max_temp',
 'max_hr',
 'max_resp',
 'max_o2sat',
 'max_sbp',
 'max_dbp',
 'min_temp',
 'min_hr',
 'min_resp',
 'min_o2sat',
 'min_sbp',
 'min_dbp',
 'avg_temp',
 'avg_hr',
 'avg_resp',
 'avg_o2sat',
 'avg_sbp',
 'avg_dbp']

In [22]:
con=list(result_df.columns[2:20])
for i in tqdm(con):
    result_df['inpute_'+i] = result_df.apply(lambda row: 1 if pd.isna(row[i]) else 0, axis=1)

100%|██████████| 18/18 [03:23<00:00, 11.28s/it]


In [556]:
#result_df.to_csv("pre_ED_v0.2_231221.csv")

In [23]:
result_df
result_df.iloc[:, 2:20]

,max_temp,max_hr,max_resp,max_o2sat,max_sbp,max_dbp,min_temp,min_hr,min_resp,min_o2sat,min_sbp,min_dbp,avg_temp,avg_hr,avg_resp,avg_o2sat,avg_sbp,avg_dbp
0,37.111111,96.0,18.0,93.0,160.0,54.0,37.111111,96.0,18.0,93.0,160.0,54.0,37.111111,96.0,18.0,93.0,160.0,54.0
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,80.0,13.0,99.0,112.0,44.0,NaN,80.0,13.0,99.0,112.0,44.0,NaN,80.0,13.0,99.0,112.0,44.0
4,37.000000,88.0,16.0,100.0,135.0,51.0,37.000000,88.0,16.0,100.0,135.0,51.0,37.000000,88.0,16.0,100.0,135.0,51.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1777442,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1777443,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1777444,36.666667,107.0,20.0,99.0,139.0,90.0,36.666667,107.0,20.0,99.0,139.0,90.0,36.666667,107.0,20.0,99.0,139.0,90.0
1777445,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
columns_to_fill = result_df.columns[2:20]  # Adjust the column range accordingly

result_df[columns_to_fill] = result_df.groupby('stay_id')[columns_to_fill].transform(lambda x: x.ffill())

In [25]:
result_df[result_df.isnull().any(axis=1)]['stay_id'].nunique()

15780

In [26]:
result_df=result_df[~result_df['stay_id'].isin(result_df[result_df.isnull().any(axis=1)]['stay_id'].unique())]

In [27]:
result_df.reset_index(drop=True,inplace=True)

In [28]:
result_df

,stay_id,time,max_temp,max_hr,max_resp,max_o2sat,max_sbp,max_dbp,min_temp,min_hr,min_resp,min_o2sat,min_sbp,min_dbp,avg_temp,avg_hr,avg_resp,avg_o2sat,avg_sbp,avg_dbp,drug_acetaminophen,drug_morphine,drug_ondansetron,drug_hydromorphone,drug_oxycodone,drug_lidocaine,drug_ibuprofen,drug_lorazepam,drug_albuterol,drug_ketorolac,drug_ceftriaxone,drug_ipratropium,drug_vancomycin,drug_bromide,drug_diazepam,drug_aspirin,drug_sodium,drug_magnesium,drug_fentanyl,drug_diphenhydramine,inpute_max_temp,inpute_max_hr,inpute_max_resp,inpute_max_o2sat,inpute_max_sbp,inpute_max_dbp,inpute_min_temp,inpute_min_hr,inpute_min_resp,inpute_min_o2sat,inpute_min_sbp,inpute_min_dbp,inpute_avg_temp,inpute_avg_hr,inpute_avg_resp,inpute_avg_o2sat,inpute_avg_sbp,inpute_avg_dbp
0,30000012,2126-02-14 21:22:00,37.111111,96.0,18.0,93.0,160.0,54.0,37.111111,96.0,18.0,93.0,160.0,54.0,37.111111,96.0,18.0,93.0,160.0,54.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,30000012,2126-02-14 22:22:00,37.111111,96.0,18.0,93.0,160.0,54.0,37.111111,96.0,18.0,93.0,160.0,54.0,37.111111,96.0,18.0,93.0,160.0,54.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
2,30000012,2126-02-14 23:22:00,37.111111,96.0,18.0,93.0,160.0,54.0,37.111111,96.0,18.0,93.0,160.0,54.0,37.111111,96.0,18.0,93.0,160.0,54.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
3,30000012,2126-02-15 00:22:00,37.111111,80.0,13.0,99.0,112.0,44.0,37.111111,80.0,13.0,99.0,112.0,44.0,37.111111,80.0,13.0,99.0,112.0,44.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0
4,30000012,2126-02-15 01:22:00,37.000000,88.0,16.0,100.0,135.0,51.0,37.000000,88.0,16.0,100.0,135.0,51.0,37.000000,88.0,16.0,100.0,135.0,51.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1696476,39999933,2128-07-20 19:42:00,36.888889,59.0,16.0,100.0,108.0,66.0,36.888889,59.0,16.0,100.0,108.0,66.0,36.888889,59.0,16.0,100.0,108.0,66.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
1696477,39999953,2152-06-22 15:08:00,36.777778,108.0,18.0,100.0,155.0,94.0,36.777778,108.0,18.0,100.0,155.0,94.0,36.777778,108.0,18.0,100.0,155.0,94.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1696478,39999953,2152-06-22 16:08:00,36.777778,108.0,18.0,100.0,155.0,94.0,36.777778,108.0,18.0,100.0,155.0,94.0,36.777778,108.0,18.0,100.0,155.0,94.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
1696479,39999953,2152-06-22 17:08:00,36.777778,108.0,18.0,100.0,155.0,94.0,36.777778,108.0,18.0,100.0,155.0,94.0,36.777778,108.0,18.0,100.0,155.0,94.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1


In [29]:
df

,stay_id,time,gender,race,arrival_transport,result,temperature,heartrate,resprate,o2sat,sbp,dbp,acuity,chief_abdpain,chief_transfer,chief_chestpain,chief_pfall,chief_dyspnea,chief_fever,chief_headache,chief_backpain,chief_weakness,chief_dizziness,chief_cough,chief_woundeval,chief_mvc,chief_etoh,chief_altered_mental_status,first_time,last_time,drug_acetaminophen,drug_morphine,drug_ondansetron,drug_hydromorphone,drug_oxycodone,drug_lidocaine,drug_ibuprofen,drug_lorazepam,drug_albuterol,drug_ketorolac,drug_ceftriaxone,drug_ipratropium,drug_vancomycin,drug_bromide,drug_diazepam,drug_aspirin,drug_sodium,drug_magnesium,drug_fentanyl,drug_diphenhydramine,Comorbidity_AIDS_H1V,Comorbidity_Alcohol_abuse,Comorbidity_Blood_loss_anemia,Comorbidity_Chronic_pulmonary_disease,Comorbidity_Coagulopathy,Comorbidity_Deficiency_anemia,Comorbidity_Depression,Comorbidity_Diabetes_complicated,Comorbidity_Diabetes_uncomplicated,Comorbidity_Drug_abuse,Comorbidity_Fluid_electrolyte_disorders,Comorbidity_Hypothyroidism,Comorbidity_Liver_disease,Comorbidity_Lymphoma,Comorbidity_Metastatic_cancer,Comorbidity_Obesity,Comorbidity_Other_neurological_disorders,Comorbidity_Paralysis,Comorbidity_Pepticulcer_disease,Comorbidity_Peripheral_vascular,Comorbidity_Psychoses,Comorbidity_Pulmonary_circulation,Comorbidity_Renal_failure,Comorbidity_Rheumatoid_arthritis_collagen_vascular_diseases,Comorbidity_Solid_tumor_without_metastasis,Comorbidity_Weight_loss,Comorbidity_cardiac,Comorbidity_chf,Comorbidity_hypertension_complicated,Comorbidity_hypertension_uncomplicated,Comorbidity_valvular_disease,time_diff
0,30000012,2126-02-14 20:22:00,F,White,AMBULANCE,hos,NaN,NaN,NaN,NaN,NaN,NaN,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2126-02-14 20:22:00,2126-02-15 01:59:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaT
1,30000012,2126-02-14 20:22:00,F,White,AMBULANCE,hos,37.111111,96.0,18.0,93.0,160.0,54.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2126-02-14 20:22:00,2126-02-15 01:59:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0 days 00:00:00
2,30000012,2126-02-14 20:23:00,F,White,AMBULANCE,hos,37.111111,96.0,18.0,93.0,160.0,54.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2126-02-14 20:22:00,2126-02-15 01:59:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0 days 00:01:00
3,30000012,2126-02-14 23:43:00,F,White,AMBULANCE,hos,NaN,80.0,13.0,99.0,112.0,44.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2126-02-14 20:22:00,2126-02-15 01:59:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0 days 03:20:00
4,30000012,2126-02-15 00:50:00,F,White,AMBULANCE,hos,37.000000,88.0,16.0,100.0,135.0,51.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2126-02-14 20:22:00,2126-02-15 01:59:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0 days 01:07:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2407617,39999961,2145-05-16 19:56:00,F,Bl

In [30]:
df.columns[50:-1].tolist()

['Comorbidity_AIDS_H1V',
 'Comorbidity_Alcohol_abuse',
 'Comorbidity_Blood_loss_anemia',
 'Comorbidity_Chronic_pulmonary_disease',
 'Comorbidity_Coagulopathy',
 'Comorbidity_Deficiency_anemia',
 'Comorbidity_Depression',
 'Comorbidity_Diabetes_complicated',
 'Comorbidity_Diabetes_uncomplicated',
 'Comorbidity_Drug_abuse',
 'Comorbidity_Fluid_electrolyte_disorders',
 'Comorbidity_Hypothyroidism',
 'Comorbidity_Liver_disease',
 'Comorbidity_Lymphoma',
 'Comorbidity_Metastatic_cancer',
 'Comorbidity_Obesity',
 'Comorbidity_Other_neurological_disorders',
 'Comorbidity_Paralysis',
 'Comorbidity_Pepticulcer_disease',
 'Comorbidity_Peripheral_vascular',
 'Comorbidity_Psychoses',
 'Comorbidity_Pulmonary_circulation',
 'Comorbidity_Renal_failure',
 'Comorbidity_Rheumatoid_arthritis_collagen_vascular_diseases',
 'Comorbidity_Solid_tumor_without_metastasis',
 'Comorbidity_Weight_loss',
 'Comorbidity_cardiac',
 'Comorbidity_chf',
 'Comorbidity_hypertension_complicated',
 'Comorbidity_hypertension_

In [31]:
static_df=df[['stay_id','gender','race','arrival_transport','result','acuity']+df.columns[13:30].tolist()+df.columns[50:-1].tolist()]

In [32]:
triage=pd.read_csv('triage_240308.csv')

In [33]:
triage

,stay_id,temperature,heartrate,resprate,o2sat,sbp,dbp,acuity,abdpain,transfer,chestpain,pfall,dyspnea,fever,headache,backpain,weakness,dizziness,cough,woundeval,mvc,etoh,altered_mental_status
0,32952584,36.555556,87.0,14.0,97.0,71.0,43.0,2.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,33258284,36.888889,70.0,16.0,97.0,106.0,63.0,3.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,35968195,37.444444,105.0,18.0,96.0,106.0,57.0,3.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,38112554,37.166667,88.0,18.0,97.0,116.0,88.0,3.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,39399961,37.055556,77.0,16.0,98.0,96.0,50.0,2.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
425082,37972930,36.666667,91.0,16.0,99.0,148.0,90.0,2.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
425083,30712109,36.722222,83.0,18.0,100.0,107.0,75.0,2.0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0
425084,32917002,35.888889,112.0,18.0,100.0,110.0,82.0,2.0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0
425085,32002659,37.500000,81.0,10.0,100.0,93.0,55.0,2.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1


In [34]:
triage=triage[triage.columns[0:7]]

In [35]:
static_df_1=pd.merge(static_df,triage,on='stay_id',how='left')

In [36]:
static_df_2=static_df_1.drop_duplicates()

In [37]:
static_df_2.reset_index(inplace=True,drop=True)

In [38]:
static_df_2

,stay_id,gender,race,arrival_transport,result,acuity,chief_abdpain,chief_transfer,chief_chestpain,chief_pfall,chief_dyspnea,chief_fever,chief_headache,chief_backpain,chief_weakness,chief_dizziness,chief_cough,chief_woundeval,chief_mvc,chief_etoh,chief_altered_mental_status,first_time,last_time,Comorbidity_AIDS_H1V,Comorbidity_Alcohol_abuse,Comorbidity_Blood_loss_anemia,Comorbidity_Chronic_pulmonary_disease,Comorbidity_Coagulopathy,Comorbidity_Deficiency_anemia,Comorbidity_Depression,Comorbidity_Diabetes_complicated,Comorbidity_Diabetes_uncomplicated,Comorbidity_Drug_abuse,Comorbidity_Fluid_electrolyte_disorders,Comorbidity_Hypothyroidism,Comorbidity_Liver_disease,Comorbidity_Lymphoma,Comorbidity_Metastatic_cancer,Comorbidity_Obesity,Comorbidity_Other_neurological_disorders,Comorbidity_Paralysis,Comorbidity_Pepticulcer_disease,Comorbidity_Peripheral_vascular,Comorbidity_Psychoses,Comorbidity_Pulmonary_circulation,Comorbidity_Renal_failure,Comorbidity_Rheumatoid_arthritis_collagen_vascular_diseases,Comorbidity_Solid_tumor_without_metastasis,Comorbidity_Weight_loss,Comorbidity_cardiac,Comorbidity_chf,Comorbidity_hypertension_complicated,Comorbidity_hypertension_uncomplicated,Comorbidity_valvular_disease,temperature,heartrate,resprate,o2sat,sbp,dbp
0,30000012,F,White,AMBULANCE,hos,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2126-02-14 20:22:00,2126-02-15 01:59:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,37.111111,96.0,18.0,93.0,160.0,54.0
1,30000038,F,White,AMBULANCE,hos,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,2152-12-07 16:37:00,2152-12-07 19:55:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,36.166667,54.0,18.0,95.0,143.0,73.0
2,30000039,M,White,WALK IN,hos,3.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2165-10-06 11:47:00,2165-10-06 20:18:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,37.000000,85.0,16.0,98.0,189.0,96.0
3,30000094,F,White,AMBULANCE,ed,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2183-09-04 16:08:00,2183-09-05 00:12:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,36.722222,60.0,18.0,94.0,120.0,95.0
4,30000112,F,Other,WALK IN,ed,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2157-12-12 12:45:00,2157-12-12 15:22:00,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,37.166667,85.0,18.0,100.0,123.0,75.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
327949,39999835,F,Black,AMBULANCE,hos,2.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2137-03-19 11:49:00,2137-03-19 21:25:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,36.166667,90.0,22.0,98.0,168.0,80.0
327950,39999863,F,Black,WALK IN,hos,2.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2122-01-16 18:02:00,2122-01-16 23:01:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,36.555556,46.0,18.0,100.0,222.0,83.0
327951,39999933,M,Black,WALK IN,ed,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2128-07-20 13:42:00,2128-07-20 19:49:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,37.000000,114.0,14.0,100.0,131.0,81.0
327952,39999953,F,Other,WALK IN,ed,3.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,2152-06-

In [35]:
#static_df_2[static_df_2['stay_id']==30001392]

In [39]:
static_df_2.rename(columns={'temperature': 'tri_temperature'}, inplace=True)
static_df_2.rename(columns={'heartrate': 'tri_heartrate'}, inplace=True)
static_df_2.rename(columns={'resprate': 'tri_resprate'}, inplace=True)
static_df_2.rename(columns={'o2sat': 'tri_o2sat'}, inplace=True)
static_df_2.rename(columns={'sbp': 'tri_sbp'}, inplace=True)
static_df_2.rename(columns={'dbp': 'tri_dbp'}, inplace=True)

/tmp/ipykernel_2479917/2677442260.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  static_df_2.rename(columns={'temperature': 'tri_temperature'}, inplace=True)
/tmp/ipykernel_2479917/2677442260.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  static_df_2.rename(columns={'heartrate': 'tri_heartrate'}, inplace=True)
/tmp/ipykernel_2479917/2677442260.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  static_df_2.rename(columns={'re

In [40]:
static_df_2.columns[54:].tolist()

['tri_temperature',
 'tri_heartrate',
 'tri_resprate',
 'tri_o2sat',
 'tri_sbp',
 'tri_dbp']

In [41]:
for i in tqdm(static_df_2.columns[54:].tolist()):
    static_df_2['inpute_'+i] = static_df_2.apply(lambda row: 1 if pd.isna(row[i]) else 0, axis=1)

  0%|          | 0/6 [00:00<?, ?it/s]/tmp/ipykernel_2479917/1474588068.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  static_df_2['inpute_'+i] = static_df_2.apply(lambda row: 1 if pd.isna(row[i]) else 0, axis=1)
 17%|█▋        | 1/6 [00:01<00:09,  1.92s/it]/tmp/ipykernel_2479917/1474588068.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  static_df_2['inpute_'+i] = static_df_2.apply(lambda row: 1 if pd.isna(row[i]) else 0, axis=1)
 33%|███▎      | 2/6 [00:03<00:07,  1.91s/it]/tmp/ipykernel_2479917/14

In [42]:
static_df_2.iloc[:,54:61]

,tri_temperature,tri_heartrate,tri_resprate,tri_o2sat,tri_sbp,tri_dbp,inpute_tri_temperature
0,37.111111,96.0,18.0,93.0,160.0,54.0,0
1,36.166667,54.0,18.0,95.0,143.0,73.0,0
2,37.000000,85.0,16.0,98.0,189.0,96.0,0
3,36.722222,60.0,18.0,94.0,120.0,95.0,0
4,37.166667,85.0,18.0,100.0,123.0,75.0,0
...,...,...,...,...,...,...,...
327949,36.166667,90.0,22.0,98.0,168.0,80.0,0
327950,36.555556,46.0,18.0,100.0,222.0,83.0,0
327951,37.000000,114.0,14.0,100.0,131.0,81.0,0
327952,36.777778,108.0,18.0,100.0,155.0,94.0,0


In [43]:
static_df_2.iloc[:,54:60]=static_df_2.iloc[:,54:60].fillna(0)

In [44]:
static_df_2.isnull().sum().sum()

0

In [45]:
result_df.isnull().sum().sum()

0

In [46]:
result_df

,stay_id,time,max_temp,max_hr,max_resp,max_o2sat,max_sbp,max_dbp,min_temp,min_hr,min_resp,min_o2sat,min_sbp,min_dbp,avg_temp,avg_hr,avg_resp,avg_o2sat,avg_sbp,avg_dbp,drug_acetaminophen,drug_morphine,drug_ondansetron,drug_hydromorphone,drug_oxycodone,drug_lidocaine,drug_ibuprofen,drug_lorazepam,drug_albuterol,drug_ketorolac,drug_ceftriaxone,drug_ipratropium,drug_vancomycin,drug_bromide,drug_diazepam,drug_aspirin,drug_sodium,drug_magnesium,drug_fentanyl,drug_diphenhydramine,inpute_max_temp,inpute_max_hr,inpute_max_resp,inpute_max_o2sat,inpute_max_sbp,inpute_max_dbp,inpute_min_temp,inpute_min_hr,inpute_min_resp,inpute_min_o2sat,inpute_min_sbp,inpute_min_dbp,inpute_avg_temp,inpute_avg_hr,inpute_avg_resp,inpute_avg_o2sat,inpute_avg_sbp,inpute_avg_dbp
0,30000012,2126-02-14 21:22:00,37.111111,96.0,18.0,93.0,160.0,54.0,37.111111,96.0,18.0,93.0,160.0,54.0,37.111111,96.0,18.0,93.0,160.0,54.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,30000012,2126-02-14 22:22:00,37.111111,96.0,18.0,93.0,160.0,54.0,37.111111,96.0,18.0,93.0,160.0,54.0,37.111111,96.0,18.0,93.0,160.0,54.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
2,30000012,2126-02-14 23:22:00,37.111111,96.0,18.0,93.0,160.0,54.0,37.111111,96.0,18.0,93.0,160.0,54.0,37.111111,96.0,18.0,93.0,160.0,54.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
3,30000012,2126-02-15 00:22:00,37.111111,80.0,13.0,99.0,112.0,44.0,37.111111,80.0,13.0,99.0,112.0,44.0,37.111111,80.0,13.0,99.0,112.0,44.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0
4,30000012,2126-02-15 01:22:00,37.000000,88.0,16.0,100.0,135.0,51.0,37.000000,88.0,16.0,100.0,135.0,51.0,37.000000,88.0,16.0,100.0,135.0,51.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1696476,39999933,2128-07-20 19:42:00,36.888889,59.0,16.0,100.0,108.0,66.0,36.888889,59.0,16.0,100.0,108.0,66.0,36.888889,59.0,16.0,100.0,108.0,66.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
1696477,39999953,2152-06-22 15:08:00,36.777778,108.0,18.0,100.0,155.0,94.0,36.777778,108.0,18.0,100.0,155.0,94.0,36.777778,108.0,18.0,100.0,155.0,94.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1696478,39999953,2152-06-22 16:08:00,36.777778,108.0,18.0,100.0,155.0,94.0,36.777778,108.0,18.0,100.0,155.0,94.0,36.777778,108.0,18.0,100.0,155.0,94.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
1696479,39999953,2152-06-22 17:08:00,36.777778,108.0,18.0,100.0,155.0,94.0,36.777778,108.0,18.0,100.0,155.0,94.0,36.777778,108.0,18.0,100.0,155.0,94.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1


In [47]:
result_df_1=pd.merge(result_df,static_df_2,how='left',on='stay_id')

In [49]:
#result_df_1.to_parquet('before_label_ED_1hour.parquet',index=False)

In [4]:
import pandas as pd
result_df_1=pd.read_parquet('before_label_ED_1hour.parquet')

In [5]:
result_df_1

,stay_id,time,max_temp,max_hr,max_resp,max_o2sat,max_sbp,max_dbp,min_temp,min_hr,min_resp,min_o2sat,min_sbp,min_dbp,avg_temp,avg_hr,avg_resp,avg_o2sat,avg_sbp,avg_dbp,drug_acetaminophen,drug_morphine,drug_ondansetron,drug_hydromorphone,drug_oxycodone,drug_lidocaine,drug_ibuprofen,drug_lorazepam,drug_albuterol,drug_ketorolac,drug_ceftriaxone,drug_ipratropium,drug_vancomycin,drug_bromide,drug_diazepam,drug_aspirin,drug_sodium,drug_magnesium,drug_fentanyl,drug_diphenhydramine,inpute_max_temp,inpute_max_hr,inpute_max_resp,inpute_max_o2sat,inpute_max_sbp,inpute_max_dbp,inpute_min_temp,inpute_min_hr,inpute_min_resp,inpute_min_o2sat,inpute_min_sbp,inpute_min_dbp,inpute_avg_temp,inpute_avg_hr,inpute_avg_resp,inpute_avg_o2sat,inpute_avg_sbp,inpute_avg_dbp,gender,race,arrival_transport,result,acuity,chief_abdpain,chief_transfer,chief_chestpain,chief_pfall,chief_dyspnea,chief_fever,chief_headache,chief_backpain,chief_weakness,chief_dizziness,chief_cough,chief_woundeval,chief_mvc,chief_etoh,chief_altered_mental_status,first_time,last_time,Comorbidity_AIDS_H1V,Comorbidity_Alcohol_abuse,Comorbidity_Blood_loss_anemia,Comorbidity_Chronic_pulmonary_disease,Comorbidity_Coagulopathy,Comorbidity_Deficiency_anemia,Comorbidity_Depression,Comorbidity_Diabetes_complicated,Comorbidity_Diabetes_uncomplicated,Comorbidity_Drug_abuse,Comorbidity_Fluid_electrolyte_disorders,Comorbidity_Hypothyroidism,Comorbidity_Liver_disease,Comorbidity_Lymphoma,Comorbidity_Metastatic_cancer,Comorbidity_Obesity,Comorbidity_Other_neurological_disorders,Comorbidity_Paralysis,Comorbidity_Pepticulcer_disease,Comorbidity_Peripheral_vascular,Comorbidity_Psychoses,Comorbidity_Pulmonary_circulation,Comorbidity_Renal_failure,Comorbidity_Rheumatoid_arthritis_collagen_vascular_diseases,Comorbidity_Solid_tumor_without_metastasis,Comorbidity_Weight_loss,Comorbidity_cardiac,Comorbidity_chf,Comorbidity_hypertension_complicated,Comorbidity_hypertension_uncomplicated,Comorbidity_valvular_disease,tri_temperature,tri_heartrate,tri_resprate,tri_o2sat,tri_sbp,tri_dbp,inpute_tri_temperature,inpute_tri_heartrate,inpute_tri_resprate,inpute_tri_o2sat,inpute_tri_sbp,inpute_tri_dbp
0,30000012,2126-02-14 21:22:00,37.111111,96.0,18.0,93.0,160.0,54.0,37.111111,96.0,18.0,93.0,160.0,54.0,37.111111,96.0,18.0,93.0,160.0,54.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,F,White,AMBULANCE,hos,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2126-02-14 20:22:00,2126-02-15 01:59:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,37.111111,96.0,18.0,93.0,160.0,54.0,0,0,0,0,0,0
1,30000012,2126-02-14 22:22:00,37.111111,96.0,18.0,93.0,160.0,54.0,37.111111,96.0,18.0,93.0,160.0,54.0,37.111111,96.0,18.0,93.0,160.0,54.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,F,White,AMBULANCE,hos,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2126-02-14 20:22:00,2126-02-15 01:59:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,37.111111,96.0,18.0,93.0,160.0,54.0,0,0,0,0,0,0
2,30000012,2126-02-14 23:22:00,37.111111,96.0,18.0,93.0,160.0,54.0,37.111111,96.0,18.0,93.0,160.0,54.0,37.111111,96.0,18.0,93.0,160.0,54.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,F,White,AMBULANCE,hos,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2126-02-14 20:22:00,2126-02-15 01:59:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,37.111111,96.0,18.0,93.0,160.0,54.0,0,0,0,0,0,0
3,30000012,2126-02-15 00:22:00,37.111111,80.0,13.0,99.0,112.0,44.0,37.111111,80.0,13.0,99.0,112.0,44.0,37.111111,80.0,13.0,99.0,112.0,44.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.

In [6]:
result_df_1['time']=pd.to_datetime(result_df_1['time'])
result_df_1['last_time']=pd.to_datetime(result_df_1['last_time'])

In [7]:
def create_label(row):
    if row['last_time'] - row['time'] < pd.Timedelta(hours=1):
        if row['result'] == 'ed':
            return 1
        elif row['result'] == 'hos':
            return 2
        elif row['result'] == 'icu':
            return 3 
        elif row['result']=='ed_dead':
            return 4

    return None     
result_df_1['label_after_1hour'] = result_df_1.apply(create_label, axis=1)
result_df_1['label_after_1hour'] = result_df_1['label_after_1hour'].fillna(0)

In [8]:
def create_label(row):
    if row['last_time'] - row['time'] < pd.Timedelta(hours=2):
        if row['result'] == 'ed':
            return 1
        elif row['result'] == 'hos':
            return 2
        elif row['result'] == 'icu':
            return 3 
        elif row['result']=='ed_dead':
            return 4    
    return None     
result_df_1['label_after_2hour'] = result_df_1.apply(create_label, axis=1)
result_df_1['label_after_2hour'] = result_df_1['label_after_2hour'].fillna(0)

In [9]:
def create_label(row):
    if row['last_time'] - row['time'] < pd.Timedelta(hours=3):
        if row['result'] == 'ed':
            return 1
        elif row['result'] == 'hos':
            return 2
        elif row['result'] == 'icu':
            return 3 
        elif row['result']=='ed_dead':
            return 4
    return None     
result_df_1['label_after_3hour'] = result_df_1.apply(create_label, axis=1)
result_df_1['label_after_3hour'] = result_df_1['label_after_3hour'].fillna(0)

In [10]:
def create_label(row):
    if row['last_time'] - row['time'] < pd.Timedelta(hours=4):
        if row['result'] == 'ed':
            return 1
        elif row['result'] == 'hos':
            return 2
        elif row['result'] == 'icu':
            return 3 
        elif row['result']=='ed_dead':
            return 4
    return None     
result_df_1['label_after_4hour'] = result_df_1.apply(create_label, axis=1)
result_df_1['label_after_4hour'] = result_df_1['label_after_4hour'].fillna(0)

In [11]:
def create_label(row):
    if row['last_time'] - row['time'] < pd.Timedelta(hours=5):
        if row['result'] == 'ed':
            return 1
        elif row['result'] == 'hos':
            return 2
        elif row['result'] == 'icu':
            return 3 
        elif row['result']=='ed_dead':
            return 4
    return None     
result_df_1['label_after_5hour'] = result_df_1.apply(create_label, axis=1)
result_df_1['label_after_5hour'] = result_df_1['label_after_5hour'].fillna(0)

In [12]:
def create_label(row):
    if row['last_time'] - row['time'] < pd.Timedelta(hours=6):
        if row['result'] == 'ed':
            return 1
        elif row['result'] == 'hos':
            return 2
        elif row['result'] == 'icu':
            return 3 
        elif row['result']=='ed_dead':
            return 4
    return None     
result_df_1['label_after_6hour'] = result_df_1.apply(create_label, axis=1)
result_df_1['label_after_6hour'] = result_df_1['label_after_6hour'].fillna(0)

In [13]:
result_df_1

,stay_id,time,max_temp,max_hr,max_resp,max_o2sat,max_sbp,max_dbp,min_temp,min_hr,min_resp,min_o2sat,min_sbp,min_dbp,avg_temp,avg_hr,avg_resp,avg_o2sat,avg_sbp,avg_dbp,drug_acetaminophen,drug_morphine,drug_ondansetron,drug_hydromorphone,drug_oxycodone,drug_lidocaine,drug_ibuprofen,drug_lorazepam,drug_albuterol,drug_ketorolac,drug_ceftriaxone,drug_ipratropium,drug_vancomycin,drug_bromide,drug_diazepam,drug_aspirin,drug_sodium,drug_magnesium,drug_fentanyl,drug_diphenhydramine,inpute_max_temp,inpute_max_hr,inpute_max_resp,inpute_max_o2sat,inpute_max_sbp,inpute_max_dbp,inpute_min_temp,inpute_min_hr,inpute_min_resp,inpute_min_o2sat,inpute_min_sbp,inpute_min_dbp,inpute_avg_temp,inpute_avg_hr,inpute_avg_resp,inpute_avg_o2sat,inpute_avg_sbp,inpute_avg_dbp,gender,race,arrival_transport,result,acuity,chief_abdpain,chief_transfer,chief_chestpain,chief_pfall,chief_dyspnea,chief_fever,chief_headache,chief_backpain,chief_weakness,chief_dizziness,chief_cough,chief_woundeval,chief_mvc,chief_etoh,chief_altered_mental_status,first_time,last_time,Comorbidity_AIDS_H1V,Comorbidity_Alcohol_abuse,Comorbidity_Blood_loss_anemia,Comorbidity_Chronic_pulmonary_disease,Comorbidity_Coagulopathy,Comorbidity_Deficiency_anemia,Comorbidity_Depression,Comorbidity_Diabetes_complicated,Comorbidity_Diabetes_uncomplicated,Comorbidity_Drug_abuse,Comorbidity_Fluid_electrolyte_disorders,Comorbidity_Hypothyroidism,Comorbidity_Liver_disease,Comorbidity_Lymphoma,Comorbidity_Metastatic_cancer,Comorbidity_Obesity,Comorbidity_Other_neurological_disorders,Comorbidity_Paralysis,Comorbidity_Pepticulcer_disease,Comorbidity_Peripheral_vascular,Comorbidity_Psychoses,Comorbidity_Pulmonary_circulation,Comorbidity_Renal_failure,Comorbidity_Rheumatoid_arthritis_collagen_vascular_diseases,Comorbidity_Solid_tumor_without_metastasis,Comorbidity_Weight_loss,Comorbidity_cardiac,Comorbidity_chf,Comorbidity_hypertension_complicated,Comorbidity_hypertension_uncomplicated,Comorbidity_valvular_disease,tri_temperature,tri_heartrate,tri_resprate,tri_o2sat,tri_sbp,tri_dbp,inpute_tri_temperature,inpute_tri_heartrate,inpute_tri_resprate,inpute_tri_o2sat,inpute_tri_sbp,inpute_tri_dbp,label_after_1hour,label_after_2hour,label_after_3hour,label_after_4hour,label_after_5hour,label_after_6hour
0,30000012,2126-02-14 21:22:00,37.111111,96.0,18.0,93.0,160.0,54.0,37.111111,96.0,18.0,93.0,160.0,54.0,37.111111,96.0,18.0,93.0,160.0,54.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,F,White,AMBULANCE,hos,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2126-02-14 20:22:00,2126-02-15 01:59:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,37.111111,96.0,18.0,93.0,160.0,54.0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,2.0,2.0
1,30000012,2126-02-14 22:22:00,37.111111,96.0,18.0,93.0,160.0,54.0,37.111111,96.0,18.0,93.0,160.0,54.0,37.111111,96.0,18.0,93.0,160.0,54.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,F,White,AMBULANCE,hos,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2126-02-14 20:22:00,2126-02-15 01:59:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,37.111111,96.0,18.0,93.0,160.0,54.0,0,0,0,0,0,0,0.0,0.0,0.0,2.0,2.0,2.0
2,30000012,2126-02-14 23:22:00,37.111111,96.0,18.0,93.0,160.0,54.0,37.111111,96.0,18.0,93.0,160.0,54.0,37.111111,96.0,18.0,93.0,160.0,54.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,F,White,AMBULANCE,hos,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2126-02-14 20:22:00,2126-02-15 01:59:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,37.111111,96.0,18.0,93.0,160.0,54.0,0,0,0,0,0,0,0.0,0.0,2.0,2.0,2.0,2.0
3,30000012,21

In [14]:
 result_df_1['acuity'].value_counts()

acuity
3.0    952126
2.0    604229
1.0     73793
4.0     64974
5.0      1359
Name: count, dtype: int64

In [15]:
result_df_1=result_df_1.drop(['result'],axis='columns')
result_df_1=result_df_1.drop(['last_time'],axis='columns')
result_df_1=result_df_1.drop(['first_time'],axis='columns')

In [16]:
result_df_1=pd.get_dummies(result_df_1, columns=['gender', 'race', 'arrival_transport'])
result_df_1

,stay_id,time,max_temp,max_hr,max_resp,max_o2sat,max_sbp,max_dbp,min_temp,min_hr,min_resp,min_o2sat,min_sbp,min_dbp,avg_temp,avg_hr,avg_resp,avg_o2sat,avg_sbp,avg_dbp,drug_acetaminophen,drug_morphine,drug_ondansetron,drug_hydromorphone,drug_oxycodone,drug_lidocaine,drug_ibuprofen,drug_lorazepam,drug_albuterol,drug_ketorolac,drug_ceftriaxone,drug_ipratropium,drug_vancomycin,drug_bromide,drug_diazepam,drug_aspirin,drug_sodium,drug_magnesium,drug_fentanyl,drug_diphenhydramine,inpute_max_temp,inpute_max_hr,inpute_max_resp,inpute_max_o2sat,inpute_max_sbp,inpute_max_dbp,inpute_min_temp,inpute_min_hr,inpute_min_resp,inpute_min_o2sat,inpute_min_sbp,inpute_min_dbp,inpute_avg_temp,inpute_avg_hr,inpute_avg_resp,inpute_avg_o2sat,inpute_avg_sbp,inpute_avg_dbp,acuity,chief_abdpain,chief_transfer,chief_chestpain,chief_pfall,chief_dyspnea,chief_fever,chief_headache,chief_backpain,chief_weakness,chief_dizziness,chief_cough,chief_woundeval,chief_mvc,chief_etoh,chief_altered_mental_status,Comorbidity_AIDS_H1V,Comorbidity_Alcohol_abuse,Comorbidity_Blood_loss_anemia,Comorbidity_Chronic_pulmonary_disease,Comorbidity_Coagulopathy,Comorbidity_Deficiency_anemia,Comorbidity_Depression,Comorbidity_Diabetes_complicated,Comorbidity_Diabetes_uncomplicated,Comorbidity_Drug_abuse,Comorbidity_Fluid_electrolyte_disorders,Comorbidity_Hypothyroidism,Comorbidity_Liver_disease,Comorbidity_Lymphoma,Comorbidity_Metastatic_cancer,Comorbidity_Obesity,Comorbidity_Other_neurological_disorders,Comorbidity_Paralysis,Comorbidity_Pepticulcer_disease,Comorbidity_Peripheral_vascular,Comorbidity_Psychoses,Comorbidity_Pulmonary_circulation,Comorbidity_Renal_failure,Comorbidity_Rheumatoid_arthritis_collagen_vascular_diseases,Comorbidity_Solid_tumor_without_metastasis,Comorbidity_Weight_loss,Comorbidity_cardiac,Comorbidity_chf,Comorbidity_hypertension_complicated,Comorbidity_hypertension_uncomplicated,Comorbidity_valvular_disease,tri_temperature,tri_heartrate,tri_resprate,tri_o2sat,tri_sbp,tri_dbp,inpute_tri_temperature,inpute_tri_heartrate,inpute_tri_resprate,inpute_tri_o2sat,inpute_tri_sbp,inpute_tri_dbp,label_after_1hour,label_after_2hour,label_after_3hour,label_after_4hour,label_after_5hour,label_after_6hour,gender_F,gender_M,race_Black,race_Other,race_White,arrival_transport_AMBULANCE,arrival_transport_HELICOPTER,arrival_transport_OTHER,arrival_transport_WALK IN
0,30000012,2126-02-14 21:22:00,37.111111,96.0,18.0,93.0,160.0,54.0,37.111111,96.0,18.0,93.0,160.0,54.0,37.111111,96.0,18.0,93.0,160.0,54.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,37.111111,96.0,18.0,93.0,160.0,54.0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,2.0,2.0,True,False,False,False,True,True,False,False,False
1,30000012,2126-02-14 22:22:00,37.111111,96.0,18.0,93.0,160.0,54.0,37.111111,96.0,18.0,93.0,160.0,54.0,37.111111,96.0,18.0,93.0,160.0,54.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,37.111111,96.0,18.0,93.0,160.0,54.0,0,0,0,0,0,0,0.0,0.0,0.0,2.0,2.0,2.0,True,False,False,False,True,True,False,False,False
2,30000012,2126-02-14 23:22:00,37.111111,96.0,18.0,93.0,160.0,54.0,37.111111,96.0,18.0,93.0,160.0,54.0,37.111111,96.0,18.0,93.0,160.0,54.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,37.111111,96.0,18.0,93.0,160.0,54.0,0,0,0,0,0,0,0.0,0.0,2.0,2.0,2.0,2

In [17]:
result_df_1.iloc[:,123:]=result_df_1.iloc[:,123:].astype(int)

In [18]:
result_df_1.to_csv('last_ED_new_1hour_forward.csv',index=False)

In [20]:
result_df_1.to_parquet('last_ED_new_1hour_forward.parquet',index=False)

In [19]:
result_df_1.isnull().sum().sum()

0

# EDA

In [129]:
result_df_1

,stay_id,time,max_temp,max_hr,max_resp,max_o2sat,max_sbp,max_dbp,min_temp,min_hr,min_resp,min_o2sat,min_sbp,min_dbp,avg_temp,avg_hr,avg_resp,avg_o2sat,avg_sbp,avg_dbp,before_max_temp,before_max_hr,before_max_resp,before_max_o2sat,before_max_sbp,before_max_dbp,before_min_temp,before_min_hr,before_min_resp,before_min_o2sat,before_min_sbp,before_min_dbp,before_avg_temp,before_avg_hr,before_avg_resp,before_avg_o2sat,before_avg_sbp,before_avg_dbp,drug_Ipratropium Bromide,drug_acetaminophen,drug_albuterol,drug_aspirin,drug_azithromycin,drug_ceftriaxone,drug_ciprofloxacin,drug_diazepam,drug_diphenhydramine,drug_fentanyl,drug_furosemide,drug_heparin,drug_hydromorphone,drug_ibuprofen,drug_ketorolac,drug_lidocaine,drug_lorazepam,drug_metoclopramide,drug_morphine,drug_ondansetron,drug_oxycodone,drug_oxycodone-acetaminophen,drug_vancomycin,before_drug_Ipratropium Bromide,before_drug_acetaminophen,before_drug_albuterol,before_drug_aspirin,before_drug_azithromycin,before_drug_ceftriaxone,before_drug_ciprofloxacin,before_drug_diazepam,before_drug_diphenhydramine,before_drug_fentanyl,before_drug_furosemide,before_drug_heparin,before_drug_hydromorphone,before_drug_ibuprofen,before_drug_ketorolac,before_drug_lidocaine,before_drug_lorazepam,before_drug_metoclopramide,before_drug_morphine,before_drug_ondansetron,before_drug_oxycodone,before_drug_oxycodone-acetaminophen,before_drug_vancomycin,inpute_max_temp,inpute_max_hr,inpute_max_resp,inpute_max_o2sat,inpute_max_sbp,inpute_max_dbp,inpute_min_temp,inpute_min_hr,inpute_min_resp,inpute_min_o2sat,inpute_min_sbp,inpute_min_dbp,inpute_avg_temp,inpute_avg_hr,inpute_avg_resp,inpute_avg_o2sat,inpute_avg_sbp,inpute_avg_dbp,inpute_before_max_temp,inpute_before_max_hr,inpute_before_max_resp,inpute_before_max_o2sat,inpute_before_max_sbp,inpute_before_max_dbp,inpute_before_min_temp,inpute_before_min_hr,inpute_before_min_resp,inpute_before_min_o2sat,inpute_before_min_sbp,inpute_before_min_dbp,inpute_before_avg_temp,inpute_before_avg_hr,inpute_before_avg_resp,inpute_before_avg_o2sat,inpute_before_avg_sbp,inpute_before_avg_dbp,acuity,abdpain,fever,dizziness,chestpain,dyspnea,Comorbidity_Alcohol_abuse,Comorbidity_Chronic_pulmonary_disease,Comorbidity_Diabetes_complicated,Comorbidity_Diabetes_uncomplicated,Comorbidity_Fluid_electrolyte_disorders,Comorbidity_Other_neurological_disorders,Comorbidity_Renal_failure,Comorbidity_cardiac,Comorbidity_chf,Comorbidity_hypertension_uncomplicated,tri_temperature,tri_heartrate,tri_resprate,tri_o2sat,tri_sbp,tri_dbp,inpute_tri_temperature,inpute_tri_heartrate,inpute_tri_resprate,inpute_tri_o2sat,inpute_tri_sbp,inpute_tri_dbp,label_after_1hour,label_after_2hour,label_after_3hour,label_after_4hour,label_after_5hour,label_after_6hour,gender_F,gender_M,race_Black,race_Other,race_White,arrival_transport_AMBULANCE,arrival_transport_HELICOPTER,arrival_transport_OTHER,arrival_transport_WALK IN
0,30000012,2126-02-14 23:22:00,37.111111,96.0,18.0,93.0,160.0,54.0,37.111111,96.0,18.0,93.0,160.0,54.0,37.111111,96.000000,18.000000,93.000000,160.000000,54.000000,37.111111,96.0,18.0,93.0,160.0,54.0,37.111111,96.0,18.0,93.0,160.0,54.0,37.111111,96.000000,18.000000,93.0,160.000000,54.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,37.111111,96.0,18.0,93.0,160.0,54.0,0,0,0,0,0,0,0.0,0.0,2.0,2.0,2.0,2.0,1,0,0,0,1,1,0,0,0
1,30000012,2126-02-15 00:22:00,0.000000,80.0,13.0,99.0,112.0,44.0,0.000000,80.0,13.0,99.0,112.0,44.0,0.000000,80.000000,13.000000,99.000000,112.000000,44.000000,37.111111,96.0,18.0,93.0,160.0,54.0,37.111111,96.0,18.0,93.0,160.0,54.0,37.111111,96.000000,18.000000,93.0,160.000000,54.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0

In [134]:
result=static_df[['stay_id','result']]
result=result.drop_duplicates()

In [137]:
result_df_2=pd.merge(result_df_1,result,how='left',on='stay_id')

In [154]:
#result_df_3 = result_df_2.replace(0, np.nan)

In [167]:
result_df_2.groupby('stay_id').first().groupby('result').mean().to_csv('first_row.csv')#[['avg_temp','avg_hr','avg_resp','avg_o2sat','avg_sbp','avg_dbp']]

C:\Users\DAHS\AppData\Local\Temp\ipykernel_13680\3916880633.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  result_df_2.groupby('stay_id').first().groupby('result').mean().to_csv('first_row.csv')#[['avg_temp','avg_hr','avg_resp','avg_o2sat','avg_sbp','avg_dbp']]


In [168]:
result_df_2.groupby('stay_id').last().groupby('result').mean().to_csv('last_row.csv')#[['avg_temp','avg_hr','avg_resp','avg_o2sat','avg_sbp','avg_dbp']]

C:\Users\DAHS\AppData\Local\Temp\ipykernel_13680\922635062.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  result_df_2.groupby('stay_id').last().groupby('result').mean().to_csv('last_row.csv')#[['avg_temp','avg_hr','avg_resp','avg_o2sat','avg_sbp','avg_dbp']]
